In [33]:
PROT_LOWER = -1
PROT_UPPER = 1

curve_u = []
curve_l = []
curves = {}
parameters = dict()

In = 'f.In'
Isc = '1000*f.Isc'
i_nf = '1.05*f.In*d.i_r'
i_f = '1.2*f.In*d.i_r'
tol_im_p = 10
tol_im_m = 10
tol_ii_p = 15
tol_ii_m = 15

tol_tm_p = 20
tol_tm_m = 20
t_i_p = 0.01
t_i_m = 0.001

tol_tr_p = 20
tol_tr_m = 20
# ir_i2t = str({'BYPASS': 0, '3s @ 6Ir':6, '10s @ 6Ir':6, '3s @ 7.2Ir':7.2, '10s @ 7.2Ir':7.2}) + '[d.curve]'
# Tr = str({'BYPASS': 0, '3s @ 6Ir':3, '10s @ 6Ir':10, '3s @ 7.2Ir':3, '10s @ 7.2Ir':10}) + '[d.curve]'
ir_i2t = 'd.ir_i2t'
Tr = 'd.t_r'

select_expr_list = ['d.lt_on is True', 'd.lt_on is False']

def get_curves(Ir, I1, Tr, T_conv, Im, Tm, Ii, Ti, i_tol, t_tol, curve_type):
    kr_I2t = '{0}**2*{1}'.format(ir_i2t, Tr)
    curve_vars = {'tms'         : 1, 
                    'i_n'         : Ir, 
                    'i1'          : I1,
                    'i2'          : Im,
                    't_min'       : Tm, 
                    'n'           : 100,
                    'i_tol'       : i_tol,
                    't_tol'       : t_tol,
                    'curve_type'  : curve_type,
                    'k_i2t'       : kr_I2t,
                    }
    # M, I
    curve1 = [('point', I1, T_conv),
                ('I2T', curve_vars),
                ('point', Im, Tm),
                ('point', Ii, Tm),
                ('point', Ii, Ti),
                ('point', Isc, Ti),]
    # M, I
    curve2 = [('point', Im, T_conv),
                ('point', Im, Tm),
                ('point', Ii, Tm),
                ('point', Ii, Ti),
                ('point', Isc, Ti),]
    return curve1, curve2

# Upper curves
Ir = 'd.i_r*' + In
I1 = i_f
# Tr = 'd.t_r'
T_conv = '3600 if f.In <= 63  else 2*3600'
Ii = 'd.i_i*' + In + '*' + str((100+tol_ii_p)/100)
Ti = t_i_p
Im = 'd.i_m*{0}*{1} if d.st_on else {2}'.format(Ir, (100+tol_im_p)/100, Ii)
Tm = 'max({0}, d.t_m*{1}) if d.st_on else {0}'.format(Ti, (100+tol_tm_p)/100)
i_tol = i_f + '*100-100'
t_tol = tol_tr_p
curve_type = PROT_UPPER
curve_u1, curve_u2 = get_curves(Ir, I1, Tr, T_conv, Im, Tm, Ii, Ti, i_tol, t_tol, curve_type)
# Lower curves
Ir = 'd.i_r*' + In
I1 = i_nf
# Tr = 'd.t_r'
T_conv = '3600 if f.In <= 63  else 2*3600'
Ii = 'd.i_i*' + In + '*' + str((100-tol_ii_m)/100)
Ti = t_i_m
Im = 'd.i_m*{0}*{1} if d.st_on else {2}'.format(Ir, (100-tol_im_m)/100, Ii)
Tm = 'max({0}, d.t_m*{1}) if d.st_on else {0}'.format(Ti, (100-tol_tm_m)/100)
i_tol = i_nf + '*100-100'
t_tol = -tol_tr_m
curve_type = PROT_LOWER
curve_l1, curve_l2 = get_curves(Ir, I1, Tr, T_conv, Im, Tm, Ii, Ti, i_tol, t_tol, curve_type)

parameters = {  'head_l'    : ['Long time protection', '', '', None, '', 'heading'],
                'lt_on'     : ['Enable long time protection', '', True, None, '', 'bool'],
                'i_r'       : ['Ir', 'xIn', 1, [0.40,0.45,0.50,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.90,0.95,1], 'Long-time pickup current'],
                'ir_i2t'    : ['@6xIr/@7.2xIr', 'xIr', 6, [6,7.2], 'Current at which tripping delay occurs'],
                't_r'       : ['tr', 's', 10, [3,10], 'Long-time tripping delay @ 6xIr/7.2xIr'],
                'head_s'    : ['Short time protection', '', '', None, '', 'heading'],
                'st_on'     : ['Enable short time protection', '', True, None, '', 'bool'],
                'i_m'       : ['Isd', 'xIr', 8, [1.5,2.5,4,5.5,6.5,8], 'Short-time pickup current'],
                't_m'       : ['tsd', 's', 0, [0,0.1], 'Short-time tripping delay (@ 10xIsd for I2T=ON)'],
                'head_i'    : ['Instantaneous protection', '', '', None, '', 'heading'],
                'i_i'       : ['Ii', 'xIn', 10, [10], 'Instantaneous pickup current'],
            }

curves = {'select_expr_list': select_expr_list,
            'curve_u1': curve_u1, 'curve_l1': curve_l1,
            'curve_u2': curve_u2, 'curve_l2': curve_l2,}


In [34]:
import json, re

data = {'type'          : 'protection',
        'parameters'    : parameters,
        'data'          : curves,
        'graph_model'   : []}

with open('mtx_1_2_l.json', 'w') as fp:
    json.dump(data, fp, indent=2)
